In [1]:
# Need to install ImageAnalysis3 and run the appropriate Startup_py3.py
%run "C:\Users\cosmosyw\Documents\Softwares\ImageAnalysis3\required_files\Startup_py3.py"
# append path to import ImageAnalysis3
sys.path.append(r"C:\Users\cosmosyw\Documents\Softwares")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd

2052


# Define basic parameters

In [3]:
postanalysis_folder = r'E:\DNA_analysis\Postanalysis_MeCP2'
if not os.path.exists(postanalysis_folder):
    os.mkdir(postanalysis_folder)
postanalysis_folder

'E:\\DNA_analysis\\Postanalysis_MeCP2'

In [4]:
# Required plotting setting
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
plt.rc('font', family='serif')
plt.rc('font', serif='Arial')

from ImageAnalysis3.figure_tools import _double_col_width, _single_col_width, _font_size, _ticklabel_size,_ticklabel_width

import seaborn as sns
sns.set_context("paper", rc={"font.size":_font_size,"axes.titlesize":_font_size+1,"axes.labelsize":_font_size})   

# Load data

In [5]:
####################################### Input folders ##################################
picked_spot_folders = [r'E:\DNA_analysis\20220729_Mecp2_0724_DNA\Analysis_0507\Picked\CW_v3_sorted',
                      r'E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Picked\CW_v3_sorted',
                      r'E:\DNA_analysis\20230119_Ma_1027_DNA\Analysis_0505\Picked\CW_v3_sorted',
                      r'E:\DNA_analysis\20230316_Mb_1103_DNA\Analysis_0506\Picked\CW_v3_sorted']

In [6]:
import pandas as pd
output_df_list = []

for picked_spot_folder in picked_spot_folders:
    if os.path.exists(picked_spot_folder):
        file_list = [os.path.join(picked_spot_folder, _fl) for _fl in os.listdir(picked_spot_folder) if '.h5' in _fl]
        df_exp_list = []
        for file in file_list:
            df_exp_list.append(pd.read_hdf(file))
        if len(df_exp_list)>0:
            df_exp = pd.concat(df_exp_list)
            del(df_exp_list)
        else:
            df_exp = None
    else:
        df_exp = None

    output_df_list.append(df_exp)

In [7]:
for df_exp in output_df_list:
    if not isinstance(df_exp, type(None)):
        print(len(np.unique(df_exp['finalcellID'])))
    else:
        print(0)

6533
13585
12226
11401


## Load positions

In [8]:
# positions
# modify with global coordinates
position_filenames = [
    r"\\KOLMOGOROV\Chromatin_NAS_8\20220729_Mecp2_0724_DNA\Alignment\adjusted_translated_positions_all.txt",
    r"\\KOLMOGOROV\Chromatin_NAS_7\20220819_Mecp2_DNA_from_0809\Alignment\adjusted_translated_positions_all.txt",
    r"V:\20230119_Ma_1027_DNA\Alignment\adjusted_translated_positions_all.txt",
    r"U:\20230316_Mb_1103_DNA\Alignment\adjusted_translated_positions_all.txt"
]
positions_list = [np.loadtxt(_fl, delimiter=',') for _fl in position_filenames]

## Load MERFISH

In [9]:
import seaborn as sns
import anndata
import scanpy as sc

adata_ori = sc.read(r"C:\Users\cosmosyw\Documents\Research\Mecp2_v2\results\adata\MeCP2_preprocessed_adata_labeled.h5ad")

print(np.unique(adata_ori.obs['experiment']))

adata_ori

['0722' '0724' '0808' '0809' '1027' '1029' '1101' '1103' '1105' '1107']


AnnData object with n_obs × n_vars = 152286 × 242
    obs: 'fov', 'volume', 'center_x', 'center_y', 'min_x', 'max_x', 'min_y', 'max_y', 'experiment', 'mouse', 'age', 'n_genes_by_counts', 'total_counts', 'subclass'
    var: 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

## load codebook and RNA experiments

In [11]:
rna_exp_names = ['20220724', '20220809', '20221027', '20221103']

codebook = pd.read_csv(r'\\10.245.74.158\Chromatin_NAS_8\Exported_data\20220713-Export\merged_codebook.csv')

# Process

In [12]:
def find_num_fiber(chrom, dict_numFiber):
    return dict_numFiber[chrom]

In [13]:
df_exp.head()

,replicate,FOV,orig_cellID,finalcellID,celltype,chr,hyb,region_name,x_hat,y_hat,z_hat,sig_x,sig_y,sig_z,rowID,MeCP2_genotype,fiberidx,fiberidx_jie
0,20230316,79,Cell-52,101633580033275824559292502096896637772,Astro,chr1,0,1:3742742-3759944,1867.133900,1823.953967,43.565996,1,1,1,37129023,KO,1,1
5,20230316,79,Cell-52,101633580033275824559292502096896637772,Astro,chr1,3,1:9627926-9637875,1868.439100,1817.994000,41.672197,1,1,1,37130859,KO,1,1
9,20230316,79,Cell-52,101633580033275824559292502096896637772,Astro,chr1,7,1:16251322-16259969,1874.925550,1814.860950,37.123715,1,1,1,37129597,KO,1,1
12,20230316,79,Cell-52,101633580033275824559292502096896637772,Astro,chr1,8,1:18749330-18759924,1872.577050,1812.007600,46.780062,1,1,1,37130183,KO,1,1
17,20230316,79,Cell-52,101633580033275824559292502096896637772,Astro,chr1,12,1:21732182-21745770,1870.705967,1817.273333,41.468809,1,1,1,37130756,KO,1,1


In [14]:
# save 
from tqdm import tqdm
cellDf_filename = os.path.join(postanalysis_folder, 'CellDfList_test.pkl')

pixel_size = 108
cellType_labelName = 'subclass_label_new'
#cellType_labelName ='subclass_manual_label'

if os.path.exists(cellDf_filename):
    print(f"Directly load cell_dfs")
    cell_dfs = pickle.load(open(cellDf_filename, 'rb'))
else:
    missed_cell = []
    cell_dfs = []
    for _exp_ind, (_exp, _positions, _df) in enumerate(zip(rna_exp_names,positions_list, output_df_list)):
        
        if not isinstance(_df, type(None)):
            
            _fovs = np.unique(_df['FOV'])
            for _fov in tqdm(_fovs):
                _fov_df = _df[_df['FOV']==_fov].copy()
                # convert to micron
                _fov_df[['z_um','x_um','y_um']] = _fov_df[['z_hat','x_hat','y_hat']] * pixel_size / 1000
                # convert to global
                _fov_df[['x_um','y_um']] = _fov_df[['x_um','y_um']] + np.flipud(_positions[int(_fov)])
                # loop through cells to re-assign cell type
                _fov_df['rna_experiment'] = _exp
               
                # load cell
                for _cell_name in np.unique(_fov_df['orig_cellID']):

                    _cell_df = _fov_df[_fov_df['orig_cellID']==_cell_name].copy()
                    _cell = _cell_name.split('Cell-')[1]
                    _cell_df['cell_id'] = _cell
                    

                    _cell_df.rename({'finalcellID':'uid',
                                     'FOV':'fov_id', 
                                     #'orig_cellID':'cell_id', 
                                     'celltype':'subclass'}, inplace=True, axis=1)

                    dict_chrFiber_num = {}
                    for chrom, df_fiber in _cell_df.groupby('chr'):
                        dict_chrFiber_num [chrom] = np.max(df_fiber.fiberidx.values)
                    _cell_df['numfiber'] = _cell_df.apply(lambda x:find_num_fiber(x['chr'], dict_chrFiber_num), axis=1)

                    cell_dfs.append(_cell_df)
                    
    # save
    print(f"Saving: {cellDf_filename}")
    pickle.dump(cell_dfs, open(cellDf_filename, 'wb'))
    
print(len(cell_dfs))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:35<00:00,  2.10it/s]


Saving: E:\DNA_analysis\Postanalysis_MeCP2\CellDfList_test.pkl
43745


In [15]:
sel_cols = ['rna_experiment', 'uid', 'fov_id', 'cell_id', 'subclass',
            'chr', 'hyb', 'fiberidx', 'numfiber', 'MeCP2_genotype',
            'z_um', 'x_um', 'y_um']
sel_merged_df = pd.concat([_df[sel_cols] for _df in cell_dfs])

In [16]:
sel_merged_filename = os.path.join(postanalysis_folder, 'selected_all_cells.csv')
if not os.path.exists(sel_merged_filename):
    sel_merged_df.to_csv(sel_merged_filename, index=False)

# convert into chr_2_zxys_list 

In [17]:
chr2Zxys_filename = os.path.join(postanalysis_folder, 'all_chr2Zxys.pkl')

cellInfo_filename = os.path.join(postanalysis_folder, 'all_cellInfo.pkl')

if os.path.exists(chr2Zxys_filename):
    print(f"Loading: {chr2Zxys_filename}")
    chr2ZxysList = pickle.load(open(chr2Zxys_filename, 'rb'))
if os.path.exists(cellInfo_filename):
    print(f"Loading: {cellInfo_filename}")
    cellInfoList = pickle.load(open(cellInfo_filename, 'rb'))

In [18]:
import multiprocessing as mp
from ImageAnalysis3.io_tools.aligner import spAligner_2_chr2homologList
print(len(cell_dfs))
parallel = True

43745


In [19]:
if parallel:
    num_threads = 16
    with mp.Pool(num_threads) as aligner_pool:
        align_results = aligner_pool.starmap(spAligner_2_chr2homologList, [(_cell_df, codebook) for _cell_df in cell_dfs])
        aligner_pool.close()
        aligner_pool.join()
        aligner_pool.terminate()
    len(align_results)
    chr2ZxysList = [_r[0] for _r in align_results]
    cellInfoList = [_r[1] for _r in align_results]
else:
    # sequential version
    chr2ZxysList = []
    cellInfoList = []

    for _cell_df in tqdm(cell_dfs):
        _sel_cell_df = _cell_df[sel_cols]
        chr_2_zxys, info_dict = spAligner_2_chr2homologList(_sel_cell_df, codebook)
        chr2ZxysList.append(chr_2_zxys)
        cellInfoList.append(info_dict)

if not os.path.exists(chr2Zxys_filename):
    print(f"Saving: {chr2Zxys_filename}")
    pickle.dump(chr2ZxysList, open(chr2Zxys_filename, 'wb'))

if not os.path.exists(cellInfo_filename):
    print(f"Saving: {cellInfo_filename}")
    pickle.dump(cellInfoList, open(cellInfo_filename, 'wb'))

Saving: E:\DNA_analysis\Postanalysis_MeCP2\all_chr2Zxys.pkl
Saving: E:\DNA_analysis\Postanalysis_MeCP2\all_cellInfo.pkl


In [20]:
cellInfoList[0]

{'rna_experiment': '20220724',
 'fov_id': 3,
 'cell_id': '100',
 'subclass': 'Oligo',
 'uid': '315504059834584005882116425750755973090',
 'MeCP2_genotype': 'WT'}